In [174]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource,LabelSet
from bokeh.io import output_notebook, show
output_notebook()

import MySQLdb
import pandas as pnd

mysql_args = {
    'host'       : 'eins.0nyx.net',
    'user'       : 'yatagarasu',
    'passwd'     : 'yatagarasupw!',
    'db'         : 'yatagarasu',
    'charset'    : 'utf8mb4'
}
 
connection = MySQLdb.connect(**mysql_args)
connection.autocommit(False)

Loading BokehJS ...

In [175]:
query_string = """
    SELECT mt.kickoff_date, mt.stadium_name, mt.league, mt.status,
    ct1.name AS club_name_home, mt.club_point_home,
    ct2.name AS club_name_away, mt.club_point_away
    FROM match_tbl AS mt
    LEFT OUTER JOIN club_tbl AS ct1 
    ON mt.club_id_home = ct1.id
    LEFT OUTER JOIN club_tbl AS ct2 
    ON mt.club_id_away = ct2.id
    WHERE status = 'off'
    AND kickoff_date BETWEEN '2018-02-22' and '2019-03-22'
    AND league IN ('j1')
    ORDER BY mt.kickoff_date ASC, mt.id ASC;
"""

#pnd.read_sql(query_string, connection) 


In [176]:
query_string = """
    SELECT mslt.club, ct.name, mslt.count, mslt.score, mslt.lost
    FROM (
        SELECT
         club_id_home AS club,
         'True' AS home,
         COUNT(club_point_home) AS count,
         SUM(club_point_home) AS score,
         SUM(club_point_away) AS lost
        FROM `match_tbl`
        WHERE kickoff_date BETWEEN '2018-02-22' and '2019-03-22'
        AND league IN ('j1')
        GROUP BY club_id_home
    ) AS mslt
    LEFT OUTER JOIN club_tbl AS ct ON mslt.club = ct.id
    ORDER BY `score`  DESC, lost DESC

"""
df_score_lost = pnd.read_sql(query_string, connection) 

# ベース作成
plot = figure(title='ホーム戦での得点と失点の分布(X:得点 Y:失点)',
           x_axis_label = '得点',
           y_axis_label = '失点',
           plot_width=800,
           plot_height=600
          )

# ベースにデータを配置
plot.circle(source=df_score_lost,
         x='score',
         y='lost',
         size=4)

# データに対してラベルを付ける
labels = LabelSet(source=ColumnDataSource(df_score_lost),
                  x='score',
                  y='lost',
                  text='name',
                  level='glyph',
                  x_offset=5, 
                  y_offset=5,
                  render_mode='canvas')


plot.add_layout(labels)

# 表示
show(plot)

In [177]:
query_string = """
    SELECT mslt.club, ct.name, mslt.count, mslt.score, mslt.lost
    FROM (
        SELECT
         club_id_away AS club,
         COUNT(club_point_away) AS count,
         SUM(club_point_away) AS score,
         SUM(club_point_home) AS lost
        FROM `match_tbl`
        WHERE kickoff_date BETWEEN '2018-02-22' and '2019-03-22'
        AND league IN ('j1')
        GROUP BY club_id_away
    ) AS mslt
    LEFT OUTER JOIN club_tbl AS ct ON mslt.club = ct.id
    ORDER BY `score`  DESC, lost DESC

"""
df_score_lost = pnd.read_sql(query_string, connection) 

# ベース作成
plot = figure(title='アウェー戦での得点と失点の分布(X:得点 Y:失点)',
           x_axis_label = '得点',
           y_axis_label = '失点',
           plot_width=800,
           plot_height=600
          )

# ベースにデータを配置
plot.circle(source=df_score_lost,
         x='score',
         y='lost',
         size=4)

# データに対してラベルを付ける
labels = LabelSet(source=ColumnDataSource(df_score_lost),
                  x='score',
                  y='lost',
                  text='name',
                  level='glyph',
                  x_offset=5, 
                  y_offset=5,
                  render_mode='canvas')


plot.add_layout(labels)

# 表示
show(plot)

In [178]:
query_string = """
    SELECT mslt.club, ct.name, SUM(mslt.count) AS count, SUM(mslt.score) AS score, SUM(mslt.lost) AS lost
    FROM (
        SELECT
         club_id_home AS club,
         'True' AS home,
         COUNT(club_point_home) AS count,
         SUM(club_point_home) AS score,
         SUM(club_point_away) AS lost
        FROM `match_tbl`
        WHERE kickoff_date BETWEEN '2018-02-22' and '2019-03-22'
        AND league IN ('j1')
        GROUP BY club_id_home
        UNION
        SELECT
         club_id_away AS club,
         'False' AS home,
         COUNT(club_point_away) AS count,
         SUM(club_point_away) AS score,
         SUM(club_point_home) AS lost
        FROM `match_tbl`
        WHERE kickoff_date BETWEEN '2018-02-22' and '2019-03-22'
        AND league IN ('j1')
        GROUP BY club_id_away
    ) AS mslt
    LEFT OUTER JOIN club_tbl AS ct ON mslt.club = ct.id
    GROUP BY mslt.club, ct.name  
    ORDER BY `score`  DESC, lost DESC

"""
df_score_lost = pnd.read_sql(query_string, connection) 

# ベース作成
plot = figure(title='ホーム&アウェー合計での得点と失点の分布(X:得点 Y:失点)',
           x_axis_label = '得点',
           y_axis_label = '失点',
           plot_width=800,
           plot_height=600,
           x_range=(4, 16), 
           y_range=(1, 16)
          )

# ベースにデータを配置
plot.circle(source=df_score_lost,
         x='score',
         y='lost',
         size=4)

# データに対してラベルを付ける
labels = LabelSet(source=ColumnDataSource(df_score_lost),
                  x='score',
                  y='lost',
                  text='name',
                  level='glyph',
                  x_offset=5, 
                  y_offset=5,
                  render_mode='canvas')


plot.add_layout(labels)

# 表示
show(plot)

In [172]:
if connection:
    try:
        connection.close()
    except MySQLdb.Error as ex:
        print("Connection close failed")